In [ ]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')

# Create checkpoint directory
checkpoint_dir = '/content/drive/MyDrive/voiceflow_wavlm_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
print(f"✅ WavLM Checkpoints will be saved to: {checkpoint_dir}")

## Step 2: Install Dependencies

In [ ]:
%%capture
# Install required packages (no speechbrain needed!)
!pip install -q datasets[audio] torch torchaudio transformers accelerate

print("✅ Packages installed")

In [ ]:
# Verify GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

## Step 3: Load Pre-trained WavLM Model

In [ ]:
from transformers import Wav2Vec2FeatureExtractor, WavLMForXVector
import torch

print("Loading WavLM-Base-Plus-SV from HuggingFace...")
print("This model was trained on VoxCeleb for speaker verification")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load feature extractor and model
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained('microsoft/wavlm-base-plus-sv')
wavlm_model = WavLMForXVector.from_pretrained('microsoft/wavlm-base-plus-sv').to(device)
wavlm_model.eval()  # Freeze for feature extraction

print(f"\n✅ WavLM model loaded on {device}")
print(f"   Embedding dimension: 512")
print(f"   Model frozen (feature extraction only)")

## Step 4: Load Streaming Dataset

In [ ]:
from datasets import load_dataset, Audio

# Dataset selection
DATASET_NAME = "librispeech_asr"
print(f"Loading {DATASET_NAME} in STREAMING mode...")

# Load train split
train_dataset = load_dataset(
    DATASET_NAME,
    "clean",
    split="train.360",
    streaming=True,
    trust_remote_code=True
)

# Load validation split
val_dataset = load_dataset(
    DATASET_NAME,
    "clean",
    split="validation",
    streaming=True,
    trust_remote_code=True
)

# Cast audio column
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16000, decode=True))
val_dataset = val_dataset.cast_column("audio", Audio(sampling_rate=16000, decode=True))

print("✅ Datasets ready (streaming mode)")

## Step 5: Create Simple Classifier Model

In [ ]:
import torch.nn as nn

class WavLMClassifier(nn.Module):
    """Simple classifier on top of pre-trained WavLM embeddings."""
    
    def __init__(self, embedding_dim=512, hidden_size=256, num_speakers=2, dropout=0.3):
        super().__init__()
        
        self.classifier = nn.Sequential(
            nn.Linear(embedding_dim, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, num_speakers)
        )
        
        self.num_speakers = num_speakers
    
    def forward(self, embeddings):
        return self.classifier(embeddings)
    
    def count_parameters(self):
        return sum(p.numel() for p in self.parameters())
    
    def count_trainable_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

# Initialize classifier
model = WavLMClassifier(
    embedding_dim=512,
    hidden_size=256,
    num_speakers=2,
    dropout=0.3
).to(device)

print(f"\n{'='*60}")
print(f"Model: WavLMClassifier (Simple MLP)")
print(f"Device: {device}")
print(f"Input: 512-dim WavLM embeddings")
print(f"Parameters: {model.count_parameters() / 1e3:.1f}K")
print(f"Trainable: {model.count_trainable_parameters() / 1e3:.1f}K")
print(f"{'='*60}\n")

## Step 6: Create WavLM Streaming DataLoader

In [ ]:
import torch
import torchaudio
from torch.utils.data import DataLoader, IterableDataset
import numpy as np

class StreamingWavLMDataset(IterableDataset):
    """Streaming dataset with WavLM embedding extraction."""
    
    def __init__(self, hf_dataset, wavlm_model, feature_extractor, device, 
                 target_sr=16000, duration=3.0, max_samples=None):
        self.dataset = hf_dataset
        self.wavlm_model = wavlm_model
        self.feature_extractor = feature_extractor
        self.device = device
        self.target_sr = target_sr
        self.target_length = int(target_sr * duration)
        self.max_samples = max_samples
        
        # Pre-scan for persistent label mapping
        print("🔍 Pre-scanning dataset to create stable label mapping...")
        self.speaker_to_label = {}
        self._create_label_mapping()
        print(f"✅ Mapped {len(self.speaker_to_label)} speakers to 2 labels")
        print(f"   Label 0: {sum(1 for l in self.speaker_to_label.values() if l == 0)} speakers")
        print(f"   Label 1: {sum(1 for l in self.speaker_to_label.values() if l == 1)} speakers")
    
    def _create_label_mapping(self):
        """Pre-scan dataset once to build stable speaker-to-label mapping."""
        next_label = 0
        count = 0
        
        for sample in self.dataset:
            if self.max_samples and count >= self.max_samples:
                break
            
            speaker_id = str(sample.get('speaker_id', 0))
            if speaker_id not in self.speaker_to_label:
                self.speaker_to_label[speaker_id] = next_label
                next_label = (next_label + 1) % 2
            
            count += 1
    
    def _extract_embedding(self, audio_tensor):
        """Extract WavLM embedding from audio."""
        with torch.no_grad():
            # Convert to numpy for feature extractor
            audio_np = audio_tensor.numpy()
            
            # Extract features
            inputs = self.feature_extractor(
                audio_np,
                sampling_rate=self.target_sr,
                return_tensors="pt",
                padding=True
            )
            
            # Move to device and extract embedding
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            outputs = self.wavlm_model(**inputs)
            
            # Get embedding and move to CPU
            embedding = outputs.embeddings.squeeze(0).cpu()  # Shape: [512]
            
            return embedding
    
    def __iter__(self):
        count = 0
        for sample in self.dataset:
            if self.max_samples and count >= self.max_samples:
                break
            
            try:
                # Extract audio
                audio = torch.FloatTensor(sample['audio']['array'])
                sr = sample['audio']['sampling_rate']
                
                # Resample if needed
                if sr != self.target_sr:
                    resampler = torchaudio.transforms.Resample(sr, self.target_sr)
                    audio = resampler(audio)
                
                # Pad/crop to fixed length
                if audio.shape[0] > self.target_length:
                    start = np.random.randint(0, audio.shape[0] - self.target_length)
                    audio = audio[start:start + self.target_length]
                elif audio.shape[0] < self.target_length:
                    audio = torch.nn.functional.pad(audio, (0, self.target_length - audio.shape[0]))
                
                # Extract WavLM embedding
                embedding = self._extract_embedding(audio)
                
                # Get persistent label
                speaker_id = str(sample.get('speaker_id', 0))
                label = self.speaker_to_label.get(speaker_id, 0)
                
                yield embedding, label
                count += 1
                
            except Exception as e:
                print(f"Skipping sample: {e}")
                continue

# Create streaming datasets
print("🎵 Creating WavLM streaming datasets...")
print("⚠️ Embedding extraction takes ~5-10 minutes for 10K samples...")

train_streaming = StreamingWavLMDataset(
    train_dataset, 
    wavlm_model, 
    feature_extractor,
    device,
    max_samples=10000
)

val_streaming = StreamingWavLMDataset(
    val_dataset,
    wavlm_model,
    feature_extractor,
    device,
    max_samples=1000
)

# Create dataloaders
BATCH_SIZE = 64

def collate_fn(batch):
    embeddings, labels = zip(*batch)
    return torch.stack(embeddings), torch.LongTensor(labels)

train_loader = DataLoader(
    train_streaming,
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn,
    num_workers=0
)

val_loader = DataLoader(
    val_streaming,
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn,
    num_workers=0
)

print("✅ WavLM DataLoaders ready")
print(f"   Embedding dim: 512")
print(f"   Batch size: {BATCH_SIZE}")

## Step 7: Training Configuration

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR

TRAINING_CONFIG = {
    'num_epochs': 30,
    'learning_rate': 3e-4,
    'weight_decay': 1e-3,
    'warmup_epochs': 2,
    'save_every_n_epochs': 5,
    'validate_every_n_epochs': 1,
}

optimizer = optim.AdamW(
    model.parameters(),
    lr=TRAINING_CONFIG['learning_rate'],
    weight_decay=TRAINING_CONFIG['weight_decay']
)

scheduler = CosineAnnealingLR(
    optimizer,
    T_max=TRAINING_CONFIG['num_epochs'],
    eta_min=1e-6
)

criterion = torch.nn.CrossEntropyLoss()

print("✅ Training configuration ready")

## Step 8: Checkpoint Resume Logic

In [ ]:
import glob

def load_latest_checkpoint(checkpoint_dir, model, optimizer, scheduler=None):
    checkpoints = glob.glob(f"{checkpoint_dir}/checkpoint_epoch*.pth")
    
    if not checkpoints:
        print("No checkpoint found. Starting from scratch.")
        return 0, 0.0
    
    latest_checkpoint = max(checkpoints, key=os.path.getctime)
    print(f"Loading checkpoint: {latest_checkpoint}")
    
    checkpoint = torch.load(latest_checkpoint, map_location=device)
    
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    if scheduler and 'scheduler_state_dict' in checkpoint:
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    start_epoch = checkpoint['epoch'] + 1
    best_val_acc = checkpoint.get('best_val_acc', 0.0)
    
    print(f"✅ Resumed from epoch {checkpoint['epoch']}")
    print(f"   Best validation accuracy: {best_val_acc:.2f}%\n")
    
    return start_epoch, best_val_acc

start_epoch, best_val_acc = load_latest_checkpoint(
    checkpoint_dir, model, optimizer, scheduler
)

## Step 9: Training Loop

In [ ]:
from tqdm.notebook import tqdm
import time

def train_epoch(model, train_loader, criterion, optimizer, device, epoch):
    model.train()
    total_loss = 0.0
    correct = 0
    total = 0
    num_batches = 0
    
    pbar = tqdm(train_loader, desc=f"[WavLM] Epoch {epoch+1}/{TRAINING_CONFIG['num_epochs']}")
    
    for batch_idx, (embeddings, labels) in enumerate(pbar):
        embeddings = embeddings.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(embeddings)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        num_batches += 1
        
        pbar.set_postfix({
            'loss': total_loss / num_batches,
            'acc': 100.0 * correct / total
        })
    
    avg_loss = total_loss / num_batches
    accuracy = 100.0 * correct / total
    
    return avg_loss, accuracy


def validate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    num_batches = 0
    
    with torch.no_grad():
        for embeddings, labels in tqdm(val_loader, desc="[WavLM] Validation"):
            embeddings = embeddings.to(device)
            labels = labels.to(device)
            
            outputs = model(embeddings)
            loss = criterion(outputs, labels)
            
            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            num_batches += 1
    
    avg_loss = total_loss / num_batches
    accuracy = 100.0 * correct / total
    
    return avg_loss, accuracy


# Training loop
print("\n" + "="*60)
print("Starting WavLM Training")
print("Expected: Val Acc 80-85% (pre-trained embeddings)")
print("="*60 + "\n")

training_start = time.time()

for epoch in range(start_epoch, TRAINING_CONFIG['num_epochs']):
    epoch_start = time.time()
    
    train_loss, train_acc = train_epoch(
        model, train_loader, criterion, optimizer, device, epoch
    )
    
    val_loss, val_acc = None, None
    if (epoch + 1) % TRAINING_CONFIG['validate_every_n_epochs'] == 0:
        val_loss, val_acc = validate(model, val_loader, criterion, device)
        
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_path = f"{checkpoint_dir}/best_model.pth"
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'val_acc': val_acc,
            }, best_model_path)
            print(f"✅ New best model saved! Val Acc: {val_acc:.2f}%")
    
    scheduler.step()
    
    epoch_time = time.time() - epoch_start
    print(f"\n[WavLM] Epoch {epoch+1}/{TRAINING_CONFIG['num_epochs']} Summary:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    if val_acc is not None:
        print(f"  Val Loss:   {val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
    print(f"  Time: {epoch_time:.1f}s | LR: {scheduler.get_last_lr()[0]:.2e}\n")
    
    if (epoch + 1) % TRAINING_CONFIG['save_every_n_epochs'] == 0:
        checkpoint_path = f"{checkpoint_dir}/checkpoint_epoch{epoch+1}.pth"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'train_loss': train_loss,
            'val_acc': val_acc if val_acc else 0.0,
            'best_val_acc': best_val_acc,
        }, checkpoint_path)
        print(f"💾 Checkpoint saved: {checkpoint_path}\n")

training_time = time.time() - training_start
print("\n" + "="*60)
print("WavLM Training Complete!")
print("="*60)
print(f"Total time: {training_time/60:.1f} minutes")
print(f"Best validation accuracy: {best_val_acc:.2f}%")
print(f"Improvement over raw waveform: +{best_val_acc - 61.5:.2f}%")
print(f"Improvement over MFCC: +{best_val_acc - 57.4:.2f}%")

## Step 10: Export to ONNX

In [ ]:
# Load best model
best_model_path = f"{checkpoint_dir}/best_model.pth"
checkpoint = torch.load(best_model_path, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

print(f"Loaded best WavLM model (Val Acc: {checkpoint['val_acc']:.2f}%)\n")

# Export to ONNX
dummy_input = torch.randn(1, 512).to(device)
onnx_path = f"{checkpoint_dir}/wavlm_classifier_trained.onnx"

torch.onnx.export(
    model,
    dummy_input,
    onnx_path,
    input_names=['wavlm_embeddings'],
    output_names=['speaker_logits'],
    dynamic_axes={
        'wavlm_embeddings': {0: 'batch_size'},
        'speaker_logits': {0: 'batch_size'}
    },
    opset_version=14
)

print(f"✅ ONNX model exported: {onnx_path}")
print(f"Model size: {os.path.getsize(onnx_path) / (1024**2):.2f} MB")

from google.colab import files
files.download(onnx_path)

## 🎉 WavLM Training Complete!

**No SpeechBrain dependency issues!**

Upload this notebook to Colab and run. It should work without any compatibility errors.